<a href="https://colab.research.google.com/github/LAGISHETTYNANDITHA/Data-analysis-using-python/blob/main/daup_proj_2(rgb_256)(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

rgb 256

In [3]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels
def load_images(folder, img_size=(256, 256)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                img = cv2.resize(img, img_size)  # Resize image
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(256, 256))
X_val, y_val = load_images(val_dir, img_size=(256, 256))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")


Loaded 1815 training images and 451 validation images


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training the CNN...
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 212ms/step - accuracy: 0.7021 - loss: 0.7248 - val_accuracy: 0.8780 - val_loss: 0.2845
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.8995 - loss: 0.2501 - val_accuracy: 0.9290 - val_loss: 0.1552
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.9601 - loss: 0.1300 - val_accuracy: 0.8958 - val_loss: 0.2785
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.9523 - loss: 0.1282 - val_accuracy: 0.9579 - val_loss: 0.1075
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.9872 - loss: 0.0392 - val_accuracy: 0.9800 - val_loss: 0.0818
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.9824 - loss: 0.0337 - val_accuracy: 0.9601 - val_loss: 0.1356
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.9911 - loss: 0.0163 - val_accuracy: 0.9823 - val_loss: 0.0760
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.9974 - loss: 0.0089 - va

In [5]:
import requests
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO

# Function to load and preprocess image from a URL or local path
def preprocess_image(path, target_size=(256, 256)):
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("RGB")

    img = img.resize(target_size)  # Resize image
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_url = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"  # Replace with actual image URL or local file path
predict_image(image_url, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 883ms/step
Prediction: ✅ Fresh Meat


greyscale 256

In [6]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels (convert to grayscale)
def load_images(folder, img_size=(256, 256)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize image
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images).reshape(-1, img_size[0], img_size[1], 1), np.array(labels)  # Add channel dimension

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(256, 256))
X_val, y_val = load_images(val_dir, img_size=(256, 256))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")


Loaded 1815 training images and 451 validation images


In [10]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO
import requests

# Define dataset paths (Update paths as needed)
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load grayscale images and labels
def load_images(folder, img_size=(256, 256)):
    images, labels = [], []

    if not os.path.exists(folder):  # Check if directory exists
        raise FileNotFoundError(f"Directory not found: {folder}")

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize image
                img = img / 255.0  # Normalize
                img = np.expand_dims(img, axis=-1)  # Add channel dimension (H, W, 1)

                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(256, 256))
X_val, y_val = load_images(val_dir, img_size=(256, 256))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")

# Define CNN Model for grayscale input
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((256, 256, 1))  # Adjust input shape for grayscale
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")

# Function to preprocess grayscale image for prediction
def preprocess_image(path, target_size=(256, 256)):
    """
    Loads and preprocesses a grayscale image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("L")  # Convert to grayscale
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        img = Image.open(path).convert("L")  # Convert local file to grayscale

    img = img.resize(target_size)  # Resize
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension (H, W, 1)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage (Update with actual image path)
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"
predict_image(image_path, model)


Loaded 1815 training images and 451 validation images
Training the CNN...
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 11s 121ms/step - accuracy: 0.6886 - loss: 0.7337 - val_accuracy: 0.8027 - val_loss: 0.4326
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.8407 - loss: 0.3730 - val_accuracy: 0.8714 - val_loss: 0.2882
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.9189 - loss: 0.2151 - val_accuracy: 0.9268 - val_loss: 0.1989
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.9525 - loss: 0.1374 - val_accuracy: 0.9446 - val_loss: 0.1748
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.9713 - loss: 0.0957 - val_accuracy: 0.9268 - val_loss: 0.1816
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.9720 - loss: 0.0782 - val_accuracy: 0.9401 - val_loss: 0.1846
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.9765 - loss: 0.0561 - val_accuracy: 0.9557 - val_loss: 0.2021
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━

rgb 200

In [11]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO
import requests

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels
def load_images(folder, img_size=(200, 200)):  # Changed image size to 200x200
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                img = cv2.resize(img, img_size)  # Resize image to 200x200
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(200, 200))
X_val, y_val = load_images(val_dir, img_size=(200, 200))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")

# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((200, 200, 3))  # Changed input shape to 200x200
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")

# Function to load and preprocess image from a URL or local path
def preprocess_image(path, target_size=(200, 200)):  # Changed target size to 200x200
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("RGB")

    img = img.resize(target_size)  # Resize image to 200x200
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"  # Replace with actual image path
predict_image(image_path, model)


Loaded 1815 training images and 451 validation images
Training the CNN...
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 15s 163ms/step - accuracy: 0.7019 - loss: 0.7444 - val_accuracy: 0.8160 - val_loss: 0.4664
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.8861 - loss: 0.2905 - val_accuracy: 0.8780 - val_loss: 0.2559
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.9467 - loss: 0.1434 - val_accuracy: 0.9357 - val_loss: 0.1686
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.9689 - loss: 0.0908 - val_accuracy: 0.9667 - val_loss: 0.0920
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.9772 - loss: 0.0621 - val_accuracy: 0.9690 - val_loss: 0.1229
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.9800 - loss: 0.0521 - val_accuracy: 0.9601 - val_loss: 0.1674
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.9877 - loss: 0.0480 - val_accuracy: 0.9778 - val_loss: 0.1042
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━

greyscale 200

In [12]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO
import requests

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels (convert to grayscale)
def load_images(folder, img_size=(200, 200)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize image to 200x200
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images).reshape(-1, 200, 200, 1), np.array(labels)  # Reshape for CNN

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(200, 200))
X_val, y_val = load_images(val_dir, img_size=(200, 200))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")

# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((200, 200, 1))  # Changed input shape to (200, 200, 1) for grayscale
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")

# Function to load and preprocess image from a URL or local path (convert to grayscale)
def preprocess_image(path, target_size=(200, 200)):
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("L")  # Convert to grayscale
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("L")  # Convert to grayscale

    img = img.resize(target_size)  # Resize image to 200x200
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dimensions
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"
predict_image(image_path, model)


Loaded 1815 training images and 451 validation images
Training the CNN...
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - accuracy: 0.6690 - loss: 0.6887 - val_accuracy: 0.8625 - val_loss: 0.3858
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.8507 - loss: 0.3523 - val_accuracy: 0.9024 - val_loss: 0.2582
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9352 - loss: 0.1730 - val_accuracy: 0.9357 - val_loss: 0.1799
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9632 - loss: 0.0999 - val_accuracy: 0.9180 - val_loss: 0.2318
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.9721 - loss: 0.0768 - val_accuracy: 0.9468 - val_loss: 0.1466
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9784 - loss: 0.0522 - val_accuracy: 0.9645 - val_loss: 0.1190
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.9900 - loss: 0.0348 - val_accuracy: 0.9557 - val_loss: 0.1125
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━